In [1]:
import tensorflow as tf
import os
from pprint import pprint



In [2]:
cwd = os.getcwd()

In [3]:
tar_path = os.path.join(cwd,'dac_sample.tar.gz')

In [4]:
# tar_file = tf.keras.utils.get_file(tar_path, 'https://labs.criteo.com/wp-content/uploads/2015/04/dac_sample.tar.gz')
tar_file = tf.keras.utils.get_file(tar_path, 'http://go.criteo.net/criteo-research-kaggle-display-advertising-challenge-dataset.tar.gz')  

In [5]:
import tarfile
my_tar = tarfile.open(tar_file)
my_tar.extractall(cwd) # specify which folder to extract to
my_tar.close()

In [6]:
%ls

dac_sample.tar.gz
dac_sample.txt
deep-learning-recommendation-model-dlrm.ipynb*
deep-learning-recommendation-model-xDeepFM.ipynb
deep_CTR.ipynb
dlrm.ipynb
license.txt
readme.txt
test_tf.ipynb
test_xdeepFM-new_dataset.ipynb
test_xdeepFM.ipynb


In [7]:
file_path = os.path.join(cwd, "dac_sample.txt")

In [8]:
columns = ['label', *(f'I{i}' for i in range(1, 14)), *(f'C{i}' for i in range(1, 27))]

In [9]:
records_type =[tf.int32, *(tf.constant([0.0], dtype=tf.float32) for _ in range(1,14)), *(tf.constant(["missing"], dtype=tf.string) for _ in range (1,27))]

2022-02-09 14:57:39.496530: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2022-02-09 14:57:39.496567: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-I6J6GOV): /proc/driver/nvidia/version does not exist
2022-02-09 14:57:39.496857: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
ds = tf.data.experimental.CsvDataset(file_path, field_delim="\t", record_defaults=records_type) 

In [11]:
ds_size = sum(1 for _ in ds)

In [12]:
ds_size

100000

In [13]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [14]:
columns = [*(f'I{i}' for i in range(1, 14)), *(f'C{i}' for i in range(1, 27))]

In [15]:
EMBEDDING_SIZE = 10
NUM_FEATURES0 = len(columns)

In [16]:
def _parse_csv_row(*vals):
    real_feats = vals[1:15]
    cat_feats = vals[15:]
    class_label = vals[0]
    features = dict(zip(columns, real_feats+cat_feats))
    return features, class_label
    

In [17]:
dataset = ds.map(_parse_csv_row)

In [18]:
pprint(list(dataset.take(1)))

[({'C1': <tf.Tensor: shape=(), dtype=string, numpy=b'68fd1e64'>,
   'C10': <tf.Tensor: shape=(), dtype=string, numpy=b'a8cd5504'>,
   'C11': <tf.Tensor: shape=(), dtype=string, numpy=b'b2cb9c98'>,
   'C12': <tf.Tensor: shape=(), dtype=string, numpy=b'37c9c164'>,
   'C13': <tf.Tensor: shape=(), dtype=string, numpy=b'2824a5f6'>,
   'C14': <tf.Tensor: shape=(), dtype=string, numpy=b'1adce6ef'>,
   'C15': <tf.Tensor: shape=(), dtype=string, numpy=b'8ba8b39a'>,
   'C16': <tf.Tensor: shape=(), dtype=string, numpy=b'891b62e7'>,
   'C17': <tf.Tensor: shape=(), dtype=string, numpy=b'e5ba7672'>,
   'C18': <tf.Tensor: shape=(), dtype=string, numpy=b'f54016b9'>,
   'C19': <tf.Tensor: shape=(), dtype=string, numpy=b'21ddcdc9'>,
   'C2': <tf.Tensor: shape=(), dtype=string, numpy=b'80e26c9b'>,
   'C20': <tf.Tensor: shape=(), dtype=string, numpy=b'b1252a9d'>,
   'C21': <tf.Tensor: shape=(), dtype=string, numpy=b'07b5194c'>,
   'C22': <tf.Tensor: shape=(), dtype=string, numpy=b'missing'>,
   'C23': <tf

In [19]:
ds_train, ds_valid, ds_test = get_dataset_partitions_tf(dataset, ds_size)

In [20]:
import numpy as np
def _get_bins(ds, col):
    ds_tmp = ds.batch(128)
    quantiles = [ x/10.0 for x in range(11)]
    y = []
    for x in ds_tmp:
        y.extend(list(x[0][col].numpy())) 
    return np.quantile(y, quantiles)

In [21]:
def _get_unique(ds, col):
    ds_tmp = ds.batch(128)
    uniques = set()
    for x in ds_tmp:
        u,i = tf.unique(x[0][col])
        for v in u.numpy():
            uniques.add(v)
    return list(uniques)

In [22]:
feature_columns = []
for i in range(1, 14):
    column_name = f'I{i}'
    bins = list(np.unique(_get_bins(ds_train, column_name)))
    tmp_num = tf.feature_column.numeric_column(column_name)
    tmp_num_buck = tf.feature_column.bucketized_column(tmp_num, boundaries=bins)
    tmp_cat_embedding = tf.feature_column.embedding_column(tmp_num_buck, dimension=EMBEDDING_SIZE)
    feature_columns.append(tmp_cat_embedding)
for i in range(1, 27):
    column_name = f'C{i}'
    unique_values = _get_unique(ds_train, column_name)
    tmp_cat = tf.feature_column.categorical_column_with_vocabulary_list(column_name, unique_values)
    tmp_cat_embedding = tf.feature_column.embedding_column(tmp_cat, dimension=EMBEDDING_SIZE)
    feature_columns.append(tmp_cat_embedding)

In [23]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [24]:
ds_train = ds_train.shuffle(1024).batch(128)
ds_valid = ds_valid.shuffle(1024).batch(128)
ds_test = ds_test.shuffle(1024).batch(128)


In [29]:
class CINLayer(tf.keras.layers.Layer):
    def __init__(self, embedding_size, num_features0, num_featuresk_1, num_featuresk):
        super(CINLayer, self).__init__()
        self.embedding_size = embedding_size # d
        self.num_features0 = num_features0 # m
        self.num_featuresk_1 = num_featuresk_1 # Hk-1
        self.num_featuresk = num_featuresk # Hk
        self.w = self.add_weight(shape=[self.num_featuresk, 
                                         self.num_featuresk_1, 
                                         self.num_features0])
        
    def call(self, inputs):
        inputs_0 = inputs[:,:self.num_features0*self.embedding_size]
        inputs_k_1 = inputs[:,self.num_features0*self.embedding_size:(self.num_features0+self.num_featuresk_1)*self.embedding_size]
        pooling = inputs[:,(self.num_features0+self.num_featuresk_1)*self.embedding_size:]
        inputs_0_reshape = tf.reshape(inputs_0, shape=[-1, self.num_features0, self.embedding_size])
        inputs_k_1_reshape = tf.reshape(inputs_k_1, shape=[-1, self.num_featuresk_1, self.embedding_size])
        hadamard_result = tf.einsum('ife,ige->iefg', inputs_k_1_reshape, inputs_0_reshape)
        outputs_reshape = tf.einsum('hfg,iefg->ihe', self.w, hadamard_result)
        pooling_k = tf.reduce_sum(outputs_reshape, 2)
        pooling_concatenate = tf.concat([pooling, pooling_k], axis=1)
        outputs = tf.reshape(outputs_reshape, shape= [-1, self.num_featuresk*self.embedding_size])
        return tf.concat([inputs_0, outputs, pooling_concatenate], axis=1)
                
                
        

In [30]:
def Linear(input_size):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(input_size,)))
    model.add(tf.keras.layers.Dense(1, activation=None))
    return model
    
def CINmodel(embedding_size, num_features0, arch_CIN):
    model = tf.keras.models.Sequential()
    for m_k_1,m_k in zip(arch_CIN[:-1], arch_CIN[1:]):
#         print(f"m_k_1  {m_k_1}, m_k {m_k}")
        model.add(CINLayer(embedding_size, num_features0, m_k_1, m_k))
    return model

def DNNmodel(input_size, arch_DNN, activation='relu'):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(input_size,)))
    for units in arch_DNN:
        model.add(tf.keras.layers.Dense(units, activation=activation))
    return model

class xDeepFM(tf.keras.Model):
    def __init__(self, embedding_size, num_features0, feature_layer, arch_CIN, arch_DNN):
        super(xDeepFM, self).__init__()
        self.embedding_size = embedding_size
        self.num_features0 = num_features0
        self.feature_layer = feature_layer
        self.num_feature_last_CIN = arch_CIN[-1]
        self.model_CIN = CINmodel(embedding_size, num_features0, arch_CIN)
        self.model_DNN = DNNmodel(embedding_size*num_features0, arch_DNN)
        self.model_linear = Linear(embedding_size*num_features0+sum(arch_CIN)+arch_DNN[-1])
        
    def call(self, inputs):
        embeddings = self.feature_layer(inputs)
        inputs_0_reshape = tf.reshape(embeddings, shape=[-1, self.num_features0, self.embedding_size])
        pooling_0 = tf.reduce_sum(inputs_0_reshape, 2)
        concat_input_CIN = tf.concat([embeddings, embeddings, pooling_0], axis=1)
        concat_output_CIN = self.model_CIN(concat_input_CIN)
        poolings = concat_output_CIN[:,(self.num_features0+self.num_feature_last_CIN)*self.embedding_size:]
        outs_DNN = self.model_DNN(embeddings)
        inputs_linear = tf.concat([embeddings, poolings, outs_DNN], axis=1)
        return self.model_linear(inputs_linear)


In [31]:
arch_CIN=[39,50,50]
arch_DNN=[512,256,128]

model = xDeepFM(EMBEDDING_SIZE, NUM_FEATURES0, feature_layer, arch_CIN, arch_DNN)

In [32]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(ds_train,
          validation_data=ds_valid,
          callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)
            ],
        epochs=100,
        verbose=1)

Epoch 1/100
625/625 [==============================] - 27s 35ms/step - loss: 0.4667 - accuracy: 0.7858 - val_loss: 0.4485 - val_accuracy: 0.7904
Epoch 2/100
625/625 [==============================] - 21s 33ms/step - loss: 0.4140 - accuracy: 0.8073 - val_loss: 0.4217 - val_accuracy: 0.8082
Epoch 3/100
625/625 [==============================] - 21s 33ms/step - loss: 0.3215 - accuracy: 0.8558 - val_loss: 0.4019 - val_accuracy: 0.8194
Epoch 4/100
625/625 [==============================] - 21s 33ms/step - loss: 0.2042 - accuracy: 0.9149 - val_loss: 0.3949 - val_accuracy: 0.8345
Epoch 5/100
625/625 [==============================] - 21s 33ms/step - loss: 0.1213 - accuracy: 0.9529 - val_loss: 0.4001 - val_accuracy: 0.8427
Epoch 6/100
625/625 [==============================] - 23s 36ms/step - loss: 0.0746 - accuracy: 0.9723 - val_loss: 0.4076 - val_accuracy: 0.8540
Epoch 7/100
625/625 [==============================] - 21s 33ms/step - loss: 0.0518 - accuracy: 0.9819 - val_loss: 0.4227 - val_ac

In [33]:
loss, accuracy = model.evaluate(ds_test)
print("Accuracy", accuracy)

79/79 [==============================] - 4s 10ms/step - loss: 0.4596 - accuracy: 0.8126
Accuracy 0.8126000165939331


In [34]:
model.summary()

Model: "x_deep_fm_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_features (DenseFeatur  multiple                 2028930   
 es)                                                             
                                                                 
 sequential_3 (Sequential)   (None, 1029)              173550    
                                                                 
 sequential_4 (Sequential)   (None, 128)               364416    
                                                                 
 sequential_5 (Sequential)   (None, 1)                 658       
                                                                 
Total params: 2,567,554
Trainable params: 2,567,554
Non-trainable params: 0
_________________________________________________________________
